In [1]:
cd ../../..

/home/msi/projects/diplomka


In [2]:
import pandas as pd
import glob
from datetime import datetime, timedelta
import networkx as nx
import math
import numpy as np
import matplotlib.pyplot as plt
from itertools import count
import seaborn as sns
import netgraph as ng
from collections import defaultdict
from hashlib import sha1
import json
import pandas as pd
import netgraph as ng

In [3]:
from traced import TraceMonitor, SiteTraceMonitor

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

np.printoptions(precision=2)

sns.set_style("darkgrid")
sns.set_context("paper")

# Get files

In [5]:
dataset = 'FZK_LCCG2-INFN_T1'
# dataset = 'INFN_T1-CSCS_LCG2'
dataset = 'CA_SFU_T2-CSCS_LCG2'
# dataset = 'SARA_MATRIX-CSCS_LCG2'
# dataset = 'CA-VICTORIA-WESTGRID-T2_CSCS-LCG2'
# dataset = 'NDGF_T1-CSCS_LCG2'
# dataset = 'BNL_ATLAS-CSCS_LCG2'
# dataset = 'Nebraska-CSCS_LCG2'
# dataset = 'GLOW-CSCS_LCG2'
# dataset = 'AGLT2-CSCS_LCG2'

In [6]:
# files = sorted(glob.glob("data/sites/CA_SFU_T2-CSCS_LCG2/*.json"))
files = sorted(glob.glob(f"data/sites/{dataset}/*.json"))
len(files)


44635

In [7]:
with open(files[0], 'r') as f:
    data = json.load(f)

In [8]:
src, dest = data['src'], data['dest']

# Processing

In [9]:
files = sorted(
    # glob.glob("data/out/FZK_LCCG2-INF_T1/*.json")
    files
    )

In [10]:
%load_ext snakeviz

In [11]:
with open(files[12], "r") as f:
    json_data = json.load(f)

In [12]:
src_site, dest_site = dataset.split('-')

In [13]:
stm = SiteTraceMonitor(src_site, dest_site)

In [14]:
stm.src

'CA_SFU_T2'

In [15]:
## NOTE: I substracted index from ttl
## NOTE: Does TTL anomalies occur in same path, do they correlate with RTT anomalies?

In [16]:
stm.process(files)

100%|██████████| 44635/44635 [00:48<00:00, 917.88it/s] 


In [17]:
json_data['ttls']

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16]

In [18]:
import random
pair = random.choice(list(stm.trace_models.keys()))

In [19]:
# for model in stm.trace_models[pair].rtt_poissons.values():
#     model.plot()

In [20]:
stm.trace_models[pair].

SyntaxError: invalid syntax (1426321339.py, line 1)

In [ ]:
data = stm.trace_models[pair].hops_ip_model.path_probs.to_frame()
data = data[filter(lambda x: x[0]== 'p', data.columns)]

In [ ]:
keys = iter(list(stm.trace_models[pair].rtt_models.keys()))

In [ ]:
key =  next(keys)

In [ ]:
keys = random.choice(list(stm.trace_models[pair].rtt_models.keys()))

In [ ]:
fig = plt.figure(figsize=(12, 6))

stm.trace_models[pair].destination_reached_model.plot()

In [ ]:
stm.trace_models[pair].n_hops_model.plot()

In [ ]:
stm.trace_models[pair].destination_reached_model.to_frame()['success_prob'].diff().apply(lambda x: np.log1p(-x)).plot()

In [ ]:
fig = plt.figure(figsize=(12, 6))
stm.trace_models[pair].rtt_models[key].plot(fig.gca())

In [ ]:
fig = plt.figure(figsize=(12, 6))
stm.trace_models[pair].hops_ip_model.node_prob_edge_prob.plot(fig.gca())

In [ ]:
fig = plt.figure(figsize=(12, 6))
stm.trace_models[pair].hops_asn_model.node_prob_edge_prob.plot(fig.gca())

In [ ]:
fig = plt.figure(figsize=(12, 6))
stm.trace_models[pair].path_probs.plot(fig.gca())
fig.gca().get_legend().remove() 

In [ ]:
stm.trace_models[pair].ttl_poissons[key].plot()

In [ ]:
x = sorted(stm.trace_models[pair].detected.keys())

In [ ]:
detected = stm.trace_models[pair].detected

In [ ]:
import json

In [ ]:
tmp = '-'.join(dataset)


In [ ]:

class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        return super().encode(bool(obj)) \
            if isinstance(obj, np.bool_) \
            else super().default(obj)
    

In [ ]:
aaaa= detected[0]

In [ ]:
aaaa

In [ ]:
# for i in stm.trace_models[model].detected.values():
#     json.dumps(dict(stm.trace_models[model].detected))
    

In [ ]:
# for k, j in i.items():
#     try:
#         json.dumps(j)
#     except TypeError as e:
#         print(k, j)
#         pass

In [ ]:
for model in stm.trace_models:
    print('->'.join(model), len(stm.trace_models[model].detected))

    with open(f"data/out/{dataset}/{'->'.join(model)}.json", 'w') as f:
        json.dump(dict(stm.trace_models[model].detected), f)

In [ ]:
tmp = iter(x)

In [ ]:
while i:= next(tmp):
    if detected[i]['has_anomaly']:
        break
i

In [ ]:
detected[i].keys()

In [ ]:
shares = [ ]
for j in x:
    shares.append(detected[j]['anom_share'])

In [ ]:
plt.plot(x, shares)

In [ ]:
df = pd.DataFrame(shares, index=x, columns=['anom_share'])

In [ ]:
df

In [ ]:
df['mean_ip_prob'] = None 
for i in x:
    df.loc[i, 'mean_ip_prob'] = detected[i]['ip'][2]

In [ ]:
df['mean_as_prob'] = None 
for i in x:
    df.loc[i, 'mean_as_prob'] = detected[i]['asn'][2]


In [ ]:
df['mean_ttl_prob'] = None 
for i in x:
    _, prob = list(zip(*detected[i]['ttls']))
    df.loc[i, 'mean_ttl_prob'] = np.mean(prob)


In [ ]:
len(list(zip(*detected[i]['rtts'])))

In [ ]:
df['mean_rtt_pdf'] = None 
for i in x:
    _, prob, _, _, _ = list(zip(*detected[i]['rtts']))
    df.loc[i, 'mean_rtt_pdf'] = np.mean(prob)


In [ ]:
df['path_prob'] = None 

for i in x:
    df.loc[i, 'path_prob'] = detected[i]['path'][1]

In [ ]:
df['dest_reached_prob']=None
for i in x:
    df.loc[i, 'dest_reached_prob'] = detected[i]['reached'][1]


In [ ]:
detected[i]['reached'][1]

In [ ]:
df.sort_index(axis=1).corr(method='spearman').style.set_caption("correlation").background_gradient(cmap='coolwarm')

## TTL and RTT vs link length


In [ ]:
# files = sorted(glob.glob("data/sites/CA_SFU_T2-CSCS_LCG2/*.json"))
data = []


In [ ]:
for file in files:
    with open(file, 'r') as f:
        json_data = json.load(f)
    src = json_data['src']
    hops = json_data['hops']
    rtts = json_data['rtts']
    ttls = json_data['ttls']
    curr=src
    for next_hop, rtt, ttl in zip(hops, rtts, ttls):
        data.append([
            curr,
            next_hop,
            rtt,
            ttl
        ])
        curr = next_hop

In [ ]:
df2 = pd.DataFrame(data, columns=['src', 'dest', 'rtt', 'ttl'])
df2

In [ ]:
geo_files = glob.glob("data/out/geo_data/*.json")
multi_site_nodes = defaultdict(list)
node_pos = {}
src_dest = set()
all_nodes  = set()

In [ ]:
node_pos

In [ ]:
edges = defaultdict(list)
for file in geo_files:
    with open(file, "r") as f:
        data = json.load(f)
    dataset = tuple(file.split('/')[-1].split('-')[:2])
    src_dest.add(data['src'])
    src_dest.add(data['dest'])
    for node, ip in data['node_mapping'].items():
        if ip in all_nodes:
            multi_site_nodes[ip].append(data['node_pos'][node])
        else:
            all_nodes.add(ip)
            node_pos[ip] = data['node_pos'][node]
    
    edges[dataset] += data['edges']

In [ ]:
geo = pd.concat(pd.read_csv(x) for x in glob.glob("data/ip_metadata/*.csv"))

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    try:
        # Convert decimal degrees to radians
        lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

        # Haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat/2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        radius = 6371  # Radius of the Earth in kilometers
        distance = radius * c
    except (ValueError, TypeError):
        distance = np.nan
        
    return distance


In [ ]:
def get_geo_info(ip):
     ip = ip.strip()
     return node_pos[ip]


In [ ]:
get_geo_info('206.12.127.40')

In [ ]:
df2.head()

In [ ]:
haversine_distance(*get_geo_info('206.12.127.40'),*get_geo_info('206.12.127.40'))

In [ ]:
geo[geo['latitude'].isna()]

In [ ]:
df2['distance'] = np.nan

In [ ]:
def calculate_distance(row):
    distance = np.nan
    # print(get_geo_info(row['src']), get_geo_info(row['dest']))
    try:
        distance = haversine_distance(*get_geo_info(row['src']), *get_geo_info(row['dest']))
        # print(distance)
    except (ValueError, IndexError):
        distance = np.nan
    finally: 
        return distance

In [ ]:
df2.sample(n=1).apply(calculate_distance, axis=1)

In [ ]:
df2['distance'] = df2.apply(calculate_distance, axis=1)

In [ ]:
df2.dropna().corr(method='pearson', numeric_only=True)

In [ ]:
df2.dropna().plot(kind='scatter', x='log_rtt', y='sqrt_dist')

In [ ]:
df2.dropna().plot(kind='scatter', x='sqrt_dist', y='ttl')

In [ ]:
df2['sqrt_ttl'] = df2['ttl'].apply(np.sqrt)
df2['log_ttl'] = df2['ttl'].apply(np.log)

In [ ]:
df2['sqrt_dist'] = df2['distance'].apply(np.sqrt)
df2['log_dist'] = df2['distance'].apply(np.log)

In [ ]:
df2['log_rtt'] = df2['rtt'].apply(np.log)
df2['sqrt_rtt'] = df2['rtt'].apply(np.sqrt)

In [ ]:
df2.dropna().plot(kind='scatter', x='sqrt_rtt', y='sqrt_ttl')

In [ ]:
df2.corr(method='spearman', numeric_only=True)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
distances = {
    (x['src'], x['dest']): x['distance']
    for i,x in df2.iterrows()
}

In [ ]:
idk = []

for file in tqdm(files):
    with open(file, 'r') as f:
        json_data = json.load(f)
    src = json_data['src']
    hops = json_data['hops']
    rtts = json_data['rtts']
    ttls = json_data['ttls']
    curr=src
    cum_size = 0
    
    for next_hop, rtt, ttl in zip(hops, rtts, ttls):
        tmp = haversine_distance(*node_pos[curr], *node_pos[next_hop])
        if np.isnan(tmp):
            tmp = 0
        cum_size += tmp
        curr = next_hop
    idk.append([cum_size, np.max(rtts, initial=0), np.max(ttls, initial=0)])

In [ ]:
df3 = pd.DataFrame(idk, columns=['total_distance', 'total_rtt', 'total_ttl'])

In [ ]:
df3.corr(method='spearman')

In [ ]:
tmp = np.where(np.array(shares) > 0.5)
tmp[0].shape

In [ ]:
from collections import Counter
c = Counter([detected[k]['path_hash'] for k in tmp[0]])
c

In [ ]:
c2 = Counter([detected[k]['ip_hash'] for k in tmp[0]])
c2

In [ ]:
c3 = Counter([detected[k]['unique_asns'] for k in tmp[0]])
c3

In [ ]:
from pprint import pprint

In [ ]:
tmp[0]

In [ ]:
d = detected[np.random.randint(0, len(detected))]
plt.plot(d['ip'][1], label='ip probs')
plt.plot(d['asn'][1], label='asn probs')
rtt = np.array(list(zip(*d['rtts']))[1])
# normalize rtt
# rtt = (rtt - rtt.min()) / (rtt.max() - rtt.min())
# rtt = (rtt ) / (rtt.std())

ttl = np.array(list(zip(*d['ttls']))[1])

plt.plot(rtt, label='rtt')
plt.plot(ttl, label='ttl')
plt.legend()

In [ ]:
%matplotlib inline

In [ ]:
max_hops = 0
for k in tmp[0]:
    k = detected[k]
    max_hops = max(max_hops,len(k['ip'][1]))
max_hops

In [ ]:
with open(files[0], 'r') as f:
    data = json.load(f)

In [ ]:
import seaborn as sns

sns.set_style("darkgrid")
sns.set_context("paper")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.stats import norm

# Set up the figure and axis
fig, ax = plt.subplots()

plt.suptitle(f"{data['src_site']} -> {data['dest_site']}")

ax.set_xlim(-0.5, max_hops + 0.5)
ax.set_ylim(-0.05, 1.1)

a, = ax.plot([0,12], [0,0], color='blue', label='RTT prob')
b, = ax.plot([0,12], [0,0], color='red', label='TTL prob')
c, = ax.plot([0,12], [0,0], color='green', label='IP prob')
e, = ax.plot([0,12], [0,0], color='orange', label='ASN prob')
# stems, = ax.plot([0, 1], [], marker='o', linestyle = 'None', color='red')

ax.legend()

# Initialize the line to be empty
def init():
    a.set_data([], [])
    b.set_data([], [])
    c.set_data([], [])
    e.set_data([], [])
    return a,b,c,e

# Generate the frames for the animation
def update(frame):

    d = detected[frame]
    ts = pd.to_datetime(d['ts'], unit='ms')
    ax.set_title(f"Anomaly detected at {ts}, score: {d['anom_share']:0.2f}, asn_hash: {d['unique_asns'][-6:]}")

    x = np.array(range(len(d['ip'][1])))

    rtt = np.array(list(zip(*d['rtts']))[1])
    # normalize rtt
    rtt = (rtt - rtt.min()) / (rtt.max() - rtt.min())
    # rtt = (rtt - rtt.mean()) / (rtt.std())

    ttl = np.array(list(zip(*d['ttls']))[1])

    a.set_data(x, rtt)
    b.set_data(x, ttl)
    c.set_data(x, d['ip'][1])
    e.set_data(x, d['asn'][1])
    
    return a,b,c,e

# Create the animation
frames = tmp[0]  # Lambda values for each frame
ani = animation.FuncAnimation(fig, update, frames=frames, init_func=init, blit=True)

# Display the animation
plt.show()


In [ ]:
f'{data["src_site"]}_{data["dest_site"]}'

In [ ]:
ani.save(f'{data["src_site"]}_{data["dest_site"]}_anomalies.gif', fps=1, )

In [ ]:
pprint(detected[1408].keys())

In [ ]:
stm.trace_models[pair].ttl_poissons[key].to_frame()

In [ ]:
for model in list(stm.trace_models.values())[0].ttl_poissons.values():
    model.plot()

In [ ]:
ips = set()
for k, v in stm.trace_models.items():
    # fig, (ax, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    # v.hops_ip_model.plot(ax)
    # v.hops_ip_model.plot_graph(ax)
    # ax.legend()
    # ax2.legend()
    ips.update(v.hops_ip_model.nodes)

In [ ]:
list(ips)

In [ ]:
# meta_df = pd.read_csv(f"data/ip_metadata/{dataset}.csv")


In [ ]:
# ip_to_geo = meta_df[(meta_df['latitude'].notna())].set_index('ip')[['latitude', 'longitude']].apply(lambda x: np.array(tuple(x)), axis=1).to_dict()

In [ ]:
# for k in stm.trace_models:
#     with open(f"data/out/geo_data/{dataset}-{k[0]}-{k[1]}.json", "w") as f:
#         f.write(stm.trace_models[k].hops_ip_model.to_json(ip_to_geo, 1))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
stm.trace_models

In [ ]:
# for k, v in stm.trace_models.items():
#     print(k)
#     fig = plt.figure(figsize=(20, 3))
#     v.destination_reached_model.plot(fig.gca())
#     plt.show()
#     fig = plt.figure(figsize=(20, 3))
#     v.hops_ip_model.plot(fig.gca())
#     plt.show()

#     for i,j in v.rtt_models.items():
#         if j.n < 20:
#             continue
#         if j.to_frame()['anomalies'].sum() == 0:
#             continue
#         fig = plt.figure(figsize=(20, 3))
#         j.plot(fig.gca())
#         plt.legend()
#         plt.show()


In [ ]:
list(stm.trace_models.keys())

In [ ]:
ta = stm.trace_models['2001:48a8:68f7:8001:192:41:236:31', '2001:620:808:4129::15']

In [ ]:
# fig= plt.figure(figsize=(10, 10))
# ta.destination_reached_model.plot(plt.gca())

In [ ]:
# # fig, (ax1, ax2) = plt.subplots( 2, 1, figsize=(12, 5))
# fig = plt.figure(figsize=(12, 5))
# ax = plt.gca()
# ta.hops_ip_model.node_prob_edge_prob.plot(ax, title='path negative log prob')

# fig = plt.figure(figsize=(12, 5))
# ax = plt.gca()
# ta.hops_ip_model.local_anomaly_model.plot(ax, title='local negative log prob')

In [ ]:
# fig=plt.figure(figsize=(20, 6))
# ta.hops_ip_model.weighted_anomaly_model.plot(plt.gca())

In [ ]:
# fig=plt.figure(figsize=(20, 6))
# ta.hops_ip_model.node_prob_edge_prob.plot(plt.gca())

In [ ]:
# fig = plt.figure(figsize=(12, 5))
# ta.destination_reached_model.plot(plt.gca())

In [ ]:
fig = plt.figure(figsize=(12, 5))
ta.hops_asn_model.node_prob_edge_prob.plot(plt.gca())

In [ ]:
import importlib

In [ ]:
import trct.models.poisson_model

In [ ]:
trct.models.poisson_model = importlib.reload(trct.models.poisson_model)

In [ ]:
ps = trct.models.poisson_model.PoissonModel('u', 'v')

In [ ]:
for i,x in data.items():
    ps.log(i.timestamp()*1000, x)

In [ ]:
ps.to_frame()

In [ ]:
ps.plot()

In [ ]:
data

In [ ]:
for m in ta.ttl_models.values():
    data = m.to_frame()['ttls']
    if data.shape[0] < 10:
        continue
    fig = plt.figure(figsize=(12, 5))

    ps = trct.models.poisson_model.PoissonModel(m.u, m.v)
    for i,x in data.items():
        ps.log(i.timestamp()*1000, x)
    
    ps.plot()
    plt.show()

In [ ]:
ips = iter(ta.ttl_models.keys())

In [ ]:
ip = next(ips)

In [ ]:
fig = plt.figure(figsize=(20, 4))

tmp = ta.ttl_models[ip].to_frame()


probs = tmp[filter(lambda x: x.startswith('p') , tmp.columns)]
probs.plot(title=ip, ax=plt.gca())

In [ ]:
data = tmp['ttls']

In [ ]:
alphas = [1]
betas = [1]

lambdas = []
vars = []
gamma_vars = []
probs = []

In [ ]:
import math

In [ ]:
data = tmp['ttls']

In [ ]:
for d in data:
    alphas.append(alphas[-1] + d)
    betas.append(betas[-1] + 1)
    lambdas.append(alphas[-1] / betas[-1])
    gamma_vars.append(alphas[-1] / (betas[-1] ** 2))
    vars.append(lambdas[-1])
    l = lambdas[-1]
    probs.append(
        l**d * np.exp(-l) / math.factorial(d)
    )

In [ ]:
np.sqrt(gamma_vars)

In [ ]:
plt.plot(data, alpha=0.5)
plt.plot(data.index, lambdas)
lambdas = np.array(lambdas)
bound = np.sqrt(vars)

plt.fill_between(data.index,  lambdas+bound, lambdas - bound, alpha=0.6, color='red', label='95% confidence interval')


In [ ]:
plt.plot(probs)
for i in range(12, 30, 3):
    
    p_i = lambdas**i * np.exp(-lambdas) / math.factorial(i)
    plt.plot( p_i, label=f'p({i})')
plt.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import factorial

In [ ]:
for time in range(1, lambdas.shape[0], lambdas.shape[0]//20 + 1):
    lm = lambdas[time]   
    t = np.linspace(0, 30, 1000)
    d = np.exp(-lm)*np.power(lm, t)/factorial(t)
    plt.title(f'Poisson({lm:2.2f}), time={time}')
    plt.plot(t, d, 'b')
    for i in range(12, 30, 3):
        p_i = lm**i * np.exp(-lm) / math.factorial(i)
        plt.plot( i, p_i, 'ro')

    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.stats import poisson

# Set up the figure and axis
fig, ax = plt.subplots()
ax.set_xlim(0, 20)
ax.set_ylim(0, 0.3)
line, = ax.plot([], [], lw=2)
stem, = ax.scatter([], [])
stem_sup = np.array(range(12, 30, 3))


# Initialize the line to be empty
def init():
    line.set_data([], [])
    return line,

# Generate the frames for the animation
def update(frame):
    x = np.arange(0, 20)
    y = poisson.pmf(x, frame)
    line.set_data(x, y)
    p_i = frame**stem_sup * np.exp(-frame) / factorial(stem_sup)


    stem.set_data(stem_sup, p_i )

    ax.set_title('Poisson Distribution (λ = {})'.format(frame))
    return line,

# Create the animation
frames = lambdas[:100]  # Lambda values for each frame
ani = animation.FuncAnimation(fig, update, frames=frames, init_func=init, blit=True)

# Display the animation
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.stats import poisson

# Set up the figure and axis
fig, ax = plt.subplots()
ax.set_xlim(0, 40)
ax.set_ylim(0, 0.17)
line, = ax.plot([], [], lw=2)
stems, = ax.plot([], [], marker='o', linestyle = 'None', color='red')
# stems, = ax.plot([], [], line='ro')
values = np.array(range(12, 23))

# Initialize the line and stems to be empty
def init():
    line.set_data([], [])
    # stems[0].set_data([], [])
    stems.set_data([], [])
    return line, stems

# Generate the frames for the animation
def update(frame):
    x = np.arange(0, 40)
    y = poisson.pmf(x, frame)
    line.set_data(x, y)
    
    # Probability values to highlight
    prob_values = poisson.pmf(values, frame)
    
    stems.set_data(values, prob_values)
    
    ax.set_title(f'Poisson Distribution (λ = {frame:.2f})')
    return line, stems

# Create the animation
frames = lambdas[:30]  # Lambda values for each frame
ani = animation.FuncAnimation(fig, update, frames=frames, init_func=init, blit=True)

# Display the animation
plt.show()


In [ ]:
ani.save('poisson.gif', writer='imagemagick', fps=4)

In [ ]:
probs = np.array(probs)

In [ ]:
an_idx = np.where(probs < 0.05)[0]

In [ ]:
plt.plot(data)
plt.plot(data.iloc[an_idx].index, data.iloc[an_idx], 'o', color='red')
plt.plot(data.index, lambdas)

In [ ]:
p_cols = list(filter(lambda x: x.startswith('p'), tmp.columns))

In [ ]:
informations = - tmp[p_cols] * np.log(tmp[p_cols])

In [ ]:
informations['sum'] = informations.sum(axis=1)

In [ ]:
(informations['p_17']/informations['sum']).plot()

In [ ]:
(informations/informations.sum(axis=1)).plot(backend='plotly')

In [ ]:
informations.plot()

In [ ]:
informations.mean(axis=1).plot()

In [ ]:
tmp['expected'] = tmp[filter(lambda x: x[0]=='p', tmp.columns)].apply(lambda x: int( x.index[np.argmax(x)][2:]), axis=1)

In [ ]:
tmp

In [ ]:
(tmp['ttls'] - tmp['expected']).plot()

In [ ]:
informations['sum_e'] = informations.sum(axis=1)

In [ ]:
informations['e_p'] = np.nan

In [ ]:
tmp['expected'] = np.nan

In [ ]:
for i,x in tmp['ttls'].items():
    print(i,x)
    informations.loc[i, 'e_p'] = np.median(informations.loc[i, f'p_{x}'])
    tmp.loc[i, 'expected'] = tmp.loc[i, f'p_{x}']

In [ ]:
informations.sample(n=10).sort_index()

In [ ]:
e = informations.drop(columns=['sum_e', 'e_p']).min(axis=1, skipna=True)

In [ ]:
tmp['ttls'].plot()

In [ ]:
(informations['e_p']/e).plot()

In [ ]:
x=((probs.max(axis=1)/tmp['expected']).apply(np.log))#.plot()

In [ ]:
from trct.models import NormalModel

In [ ]:
nm = NormalModel('u', 'v')
for i, v in enumerate(x):
    nm.log(i,v)

In [ ]:
fig  = plt.figure(figsize=(20, 5))
nm.plot(fig.gca())

In [ ]:
xxxx = nm.to_frame().reset_index()
xxx = xxxx[xxxx['anomalies']]

In [ ]:
xxx.index

In [ ]:
tmp['marked'] = False

In [ ]:
tmp.columns

In [ ]:
tmp.iloc[xxx.index, -1] = True

In [ ]:

tmp['marked'].value_counts()

In [ ]:
tmp[['ttls', 'marked']].plot(kind='scatter', y='ttls', color='marked' )

In [ ]:
fig = plt.figure(figsize=(20, 5))
nm2.plot(fig.gca())

In [ ]:
ctr = 0
for i, x in ta.ttl_models.items():
    if ctr==4:
        break
    print(i)
    fig = plt.figure(figsize=(20, 4))
    display(x.plot(plt.gca()))
    ctr+=1

In [ ]:
# fig, (ax1, ax2) = plt.subplots( 2, 1, figsize=(12, 5))
fig = plt.figure(figsize=(12, 5))
ax = plt.gca()
ta.hops_ip_model.global_anomaly_model.plot(ax)

fig = plt.figure(figsize=(12, 5))
ax = plt.gca()
ta.hops_ip_model.local_anomaly_model.plot(ax)

In [ ]:
fig=plt.figure(figsize=(20, 6))
ta.hops_ip_model.weighted_anomaly_model.plot(plt.gca())

In [ ]:
fig=plt.figure(figsize=(20, 6))
ta.hops_ip_model.node_prob_edge_prob.plot(plt.gca())

In [ ]:
s = np.array(ta.hops_ip_model.weighted_probs)

In [ ]:
plt.plot(np.log(s))

In [ ]:
df_paths = ta.hops_ip_model.to_frame()

In [ ]:
# ta.hops_ip_model.plot_graph(plt.gca(), node_layout=final_pos2)

In [ ]:

(df_paths.groupby('paths').size().sort_values(ascending=False)/df_paths.shape[0]).plot(kind='bar')

In [ ]:
N=10

In [ ]:

paths = df_paths.groupby('paths').size().sort_values(ascending=False).head(N).index.tolist()

In [ ]:
mapper = { v:f'top {i+1}' for i, v in enumerate(df_paths.groupby('paths').size().sort_values(ascending=False).rank().index.tolist())}

In [ ]:
df_paths[df_paths['paths'].isin(paths)]['paths'].unique()

In [ ]:
df_paths['weighted_probs'].apply(lambda x: )

In [ ]:
df_paths[df_paths['paths'].isin(paths)].groupby('paths')['local_probs'].plot()
plt.legend()
# plt.legend([mapper[x] for x in df_paths[df_paths['paths'].isin(paths)]['paths'].unique()], )

In [ ]:
ta.ttl_models

In [ ]:
with open(files[43456], "r") as f:
    json_data = json.load(f)

In [ ]:
node_colors = defaultdict(lambda : 'blue')

In [ ]:
path = [ta.hops_ip_model.node_to_index[x] for x in json_data['hops']] + [0]

for node in g.nodes:
    node_colors[node] = 'blue'

for node in path:
    node_colors[node] = 'red'

node_colors[ta.hops_ip_model.node_to_index['206.12.127.40']] = 'green'
node_colors[ta.hops_ip_model.node_to_index['148.187.129.15']] =  'green'

In [ ]:

#TODO add hash of path to df

In [ ]:
node_sizes = defaultdict(lambda : 1)

data = np.array(list(ta.hops_ip_model.node_in_counts.values()))   

mx = data.max()
mn = data.min()

for k, v in ta.hops_ip_model.node_out_counts.items():
    node_sizes[ta.hops_ip_model.node_to_index[k]] = ((v-mn)/(mx-mn))*2

In [ ]:

fig=plt.figure(figsize=(20, 20))
ng.Graph(
        g,
        layout='dot',
        arrows=True,
        weighted=True,
        node_color={x:node_colors[x] for x in g.nodes}, # TODO: this based on src/dest/normal
        # node_edge_color=clors,
        node_labels={x: i for i, x in enumerate(g.nodes)},
        # node_shape={x: tier_mapping[x] for x in graph.nodes},  # so^>v<dph8
        edge_cmap="RdYlGn",
        # node_colors=node_colors,
        # edge_colors=node_colors,
        node_size={x:node_sizes[x] for x in g.nodes},
        node_layout=final_pos2,
            
            )

In [ ]:
# ta.plot()

In [ ]:
from trct.models import NormalModel

In [ ]:
max_data = (np.array(ta.ttl_models["148.187.129.15"].most_probable_ttl))


In [ ]:
m = ta.ttl_models["148.187.129.15"]

In [ ]:
df = m.to_frame()

In [ ]:
tmp = df.reset_index()
c=tmp[tmp['anomalies']]['anomalies'].index.tolist()

In [ ]:
fig = plt.figure(figsize=(26, 10))
max_data = (np.array(ta.ttl_models["148.187.129.15"].ttl_prob))
plt.plot(max_data)
index = np.where(max_data < 0.05)
plt.plot(index[0], max_data[index], 'ro', label="outliers")
plt.plot(c, max_data[c], 'ro', label="outliers")
plt.legend()
plt.title(f"Outliers = {len(index[0])/len(max_data)}")

In [ ]:
fig = plt.figure(figsize=(18, 4))
ax = plt.gca()
ta.hops_ip_model.plot(ax)
ax.legend()

In [ ]:
tmp = ta.hops_ip_model.to_frame()

In [ ]:
ts, path = zip(*ta.hops_ip_model.unique_paths)

In [ ]:
x = pd.DataFrame({'path': path, 'ts':pd.to_datetime(ts, unit='ms')})

In [ ]:
cc = x.groupby('path').agg({'ts': ['min', 'max']})

In [ ]:
from trct.models import NormalModel

In [ ]:
m = NormalModel(src, dest, )

In [ ]:
tmp['local_probs'].plot()

In [ ]:
m = NormalModel(src, dest, alpha_0=0.5, mu_0=3, sigma_0=1)

for i,x in tmp['global_probs'].apply(np.log).items(): #diff
    if(np.isnan(x)):
        continue
    m.log(i.timestamp()*1000, -x)

fig = plt.figure(figsize=(20, 6))
ax =fig.gca()
m.plot(ax)
ax.set_title("Normal Model")

In [ ]:
m = NormalModel(src, dest, alpha_0=0.5, mu_0=3, sigma_0=1)

for i,x in tmp['local_probs'].apply(np.log).items(): #diff
    if(np.isnan(x)):
        continue
    m.log(i.timestamp()*1000, -x)

fig = plt.figure(figsize=(20, 6))
m.plot(fig.gca())

In [ ]:
m = NormalModel(src, dest, alpha_0=0.5, mu_0=3, sigma_0=1)

for i,x in tmp['local_probs'].apply(np.log).items(): #diff
    if(np.isnan(x)):
        continue
    m.log(i.timestamp()*1000, -x)

fig = plt.figure(figsize=(20, 6))
m.plot(fig.gca())

In [ ]:
m = NormalModel(src, dest, alpha_0=1, mu_0=20, sigma_0=1)

for i,x in tmp['global_probs'].apply(np.log).items(): #diff
    if(np.isnan(x)):
        continue
    m.log(i.timestamp()*1000, -x)

fig = plt.figure(figsize=(20, 6))
m.plot(fig.gca())

In [ ]:
m = NormalModel(src, dest, alpha_0=3, mu_0=0, sigma_0=1)
 
for i,x in tmp['local_probs'].apply(np.log).diff().clip(-np.inf, 1).items(): #diff    
    if(np.isnan(x)):
        continue
    
    m.log(i.timestamp()*1000, -x)
fig = plt.figure(figsize=(20, 6))
m.plot(fig.gca())

In [ ]:
m = NormalModel(src, dest, alpha_0=3, mu_0=20, sigma_0=1)

for i,x in tmp['global_probs'].apply(np.log).diff().items(): #diff
    if(np.isnan(x)):
        continue
    m.log(i.timestamp()*1000, -x)

fig = plt.figure(figsize=(20, 6))
m.plot(fig.gca())

In [ ]:
tmp['local_probs'].plot()

In [ ]:
fig = plt.figure(figsize=(22, 4))
fig.gca().xaxis.set_label("p")
# (tmp['global_probs'] / tmp['local_probs']).diff().plot(ax=fig.gca())
x=(tmp['local_probs'])
x.plot(ax=fig.gca())
# (tmp['global_probs']-tmp['local_probs']).diff().plot(ax=fig.gca())
# (tmp['global_probs']).diff().plot(ax=fig.gca())
# (tmp['local_probs']).plot(ax=fig.gca())
# plt.plot(cc['ts']['min'], [0]*len(cc), 'o')
# plt.plot(cc['ts']['max'], [0]*len(cc), '>')
# (tmp['local_probs'][tmp['local_probs']<0.025]).plot(ax=fig.gca(), style='x')
# tmp[tmp['weighted_probs']<0.05]['weighted_probs'].plot(ax=fig.gca(), style='x')
x[x<(0.05)].plot(ax=fig.gca(), style='x', c='r')
# print(tmp[tmp['weighted_probs']<0.05]['weighted_probs'].shape)
print(x[x<(0.05)].shape[0]/x.shape[0])
# todo try path entropy

In [ ]:
fig = plt.figure(figsize=(22, 4))
fig.gca().xaxis.set_label("p")
# (tmp['global_probs'] / tmp['local_probs']).diff().plot(ax=fig.gca())
x=((tmp['global_probs']))
x.plot(ax=fig.gca())
# (tmp['global_probs']-tmp['local_probs']).diff().plot(ax=fig.gca())
# (tmp['global_probs']).diff().plot(ax=fig.gca())
# (tmp['local_probs']).plot(ax=fig.gca())
# plt.plot(cc['ts']['min'], [0]*len(cc), 'o')
# plt.plot(cc['ts']['max'], [0]*len(cc), '>')
# (tmp['local_probs'][tmp['local_probs']<0.025]).plot(ax=fig.gca(), style='x')
# tmp[tmp['weighted_probs']<0.05]['weighted_probs'].plot(ax=fig.gca(), style='x')
x[x<(0.05)].plot(ax=fig.gca(), style='x', c='r')
# print(tmp[tmp['weighted_probs']<0.05]['weighted_probs'].shape)
print(x[x<(0.05)].shape[0]/x.shape[0])

In [ ]:
fig = plt.figure(figsize=(22, 4))
fig.gca().xaxis.set_label("p")
# (tmp['global_probs'] / tmp['local_probs']).diff().plot(ax=fig.gca())
x=((tmp['weighted_probs']))
x.plot(ax=fig.gca())
# (tmp['global_probs']-tmp['local_probs']).diff().plot(ax=fig.gca())
# (tmp['global_probs']).diff().plot(ax=fig.gca())
# (tmp['local_probs']).plot(ax=fig.gca())
# plt.plot(cc['ts']['min'], [0]*len(cc), 'o')
# plt.plot(cc['ts']['max'], [0]*len(cc), '>')
# (tmp['local_probs'][tmp['local_probs']<0.025]).plot(ax=fig.gca(), style='x')
# tmp[tmp['weighted_probs']<0.05]['weighted_probs'].plot(ax=fig.gca(), style='x')
x[x<(0.0005)].plot(ax=fig.gca(), style='x', c='r')
# print(tmp[tmp['weighted_probs']<0.05]['weighted_probs'].shape)
print(x[x<(0.005)].shape[0]/x.shape[0])